# Threads

Create a thread

In [1]:
import threading
import time

def do_threading(some_string, some_integer):
    print("in do_threading")
    print("some_string =", some_string)
    print("some_integer =", some_integer)
    time.sleep(some_integer)
    print("do_threading exiting...")

my_thread = threading.Thread(target=do_threading, args=('some arguments', 1))   # use 'daemon=True' to create daemon...
print("start thread")                                                           # background process which will
my_thread.start()                                                               # terminate when program finishes
print("returned from start, wait for the thread to complete using join()")
my_thread.join()
print("returned from join")

start thread
in do_threading
some_string = some arguments
some_integer = 1
returned from start, wait for the thread to complete using join()
do_threading exiting...
returned from join


Using a threaded class

In [2]:
class MyThread(threading.Thread):
    def __init__(self):
        self.main_lock = threading.Lock()
        self.continue_processing = True
        super().__init__()

    def get_continue_processing(self):
        with self.main_lock:
            return self.continue_processing

    def stop_processing(self):
        with self.main_lock:
            self.continue_processing = False

    def run(self):
        while self.get_continue_processing():
            print("MyThread::run: still working")
            time.sleep(0.5)
        print("MyThread::run: stopped working")

my_threaded_class = MyThread()
print("start the threaded class")
my_threaded_class.start()
print("go away and do lots of crazy stuff")
time.sleep(3)
print("ok that's all done, don't need that thread any more")
my_threaded_class.stop_processing()
print("back from stop_processing")
time.sleep(2)           # let MyThread stop before doing anything else

start the threaded class
MyThread::run: still working
go away and do lots of crazy stuff
MyThread::run: still working
MyThread::run: still working
MyThread::run: still working
MyThread::run: still working
MyThread::run: still working
ok that's all done, don't need that thread any more
back from stop_processing
MyThread::run: stopped working


Using a thread pool executor for launching multiple threads

In [3]:
import concurrent.futures

def do_some_stuff():
    time.sleep(3)
    print("done some stuff")
    return 'do_some_stuff return value'

def do_some_other_stuff():
    time.sleep(1)
    print("done some other stuff")
    return 'do_some_other_stuff return value'

executor = concurrent.futures.ThreadPoolExecutor(max_workers=2)
print("submit do_some_stuff")
stuff_future_object = executor.submit(do_some_stuff)
print("submit do_some_other_stuff")
other_future_object = executor.submit(do_some_other_stuff)
print(stuff_future_object)
print(other_future_object)
print("wait for threads to complete")
for f in concurrent.futures.as_completed([stuff_future_object, other_future_object]):
    print(f)
    print("result from this thread =", f.result())

submit do_some_stuff
submit do_some_other_stuff
<Future at 0x139956a6830 state=running>
<Future at 0x1399479b6d0 state=running>
wait for threads to complete
done some other stuff
<Future at 0x1399479b6d0 state=finished returned str>
result from this thread = do_some_other_stuff return value
done some stuff
<Future at 0x139956a6830 state=finished returned str>
result from this thread = do_some_stuff return value


Using map to launch multiple worker threads

In [4]:
def my_worker_thread(my_string):
    print("starting worker thread to calculate length of", my_string)
    if my_string == 'bob':
        time.sleep(3)
    return f"length of {my_string} is {len(my_string)}"

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    map_results = executor.map(my_worker_thread, ['bob', 'denise', 'reginald', 'sidney'])
    for result in map_results:       # will have to wait for 'bob' to finish as results provided in order given
        print(result)

starting worker thread to calculate length of bob
starting worker thread to calculate length of denise
starting worker thread to calculate length of reginald
starting worker thread to calculate length of sidney
length of bob is 3
length of denise is 6
length of reginald is 8
length of sidney is 6


Or as an alternative...

See https://stackoverflow.com/questions/20838162/how-does-threadpoolexecutor-map-differ-from-threadpoolexecutor-submit


In [5]:
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # use list comprehension, no need for map
    mapped_futures = [executor.submit(my_worker_thread, x) for x in ['bob', 'denise', 'reginald', 'sidney']]
    for f in concurrent.futures.as_completed(mapped_futures):   # get results in order of completion
        print(f.result())

starting worker thread to calculate length of bob
starting worker thread to calculate length of denise
starting worker thread to calculate length of reginald
starting worker thread to calculate length of sidney
length of reginald is 8
length of denise is 6
length of sidney is 6
length of bob is 3


Semaphores are usually shared between threads as a way to control resources

In [6]:
my_semaphore = threading.Semaphore(2)       # 2 resources

Starting thread pool with 5 threads and 2 resources

In [7]:
import random

def my_greedy_worker_thread(name, a_semaphore):
    print(f"thread {name} has started!")
    a_semaphore.acquire()
    print(f"thread {name} acquired semaphore for resource!")
    time.sleep(random.randint(1, 3))
    print(f"thread {name} finished with resource so releasing!")
    a_semaphore.release()
    return f'thread {name} is done'

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # use list comprehension, no need for map
    my_futures = [executor.submit(my_greedy_worker_thread, x, my_semaphore) for x in range(5)]
    for f in concurrent.futures.as_completed(my_futures):   # get results in order of completion
        print(f.result())

def wait_for_an_event(event, number_events):
    events_detected = 0
    while events_detected < number_events:
        print("waiting for event")
        event.wait()                    # you can specify a timeout here e.g. wait(timeout=0.5) for 500ms wait
        print("got event")
        event.clear()                   # need to clear events
        events_detected += 1
    print("all events detected, thread terminating...")

thread 0 has started!
thread 0 acquired semaphore for resource!
thread 1 has started!
thread 1 acquired semaphore for resource!
thread 2 has started!
thread 3 has started!
thread 4 has started!
thread 1 finished with resource so releasing!
thread 1 is done
thread 2 acquired semaphore for resource!
thread 0 finished with resource so releasing!
thread 3 acquired semaphore for resource!
thread 0 is done
thread 3 finished with resource so releasing!
thread 3 is done
thread 4 acquired semaphore for resource!
thread 2 finished with resource so releasing!
thread 2 is done
thread 4 finished with resource so releasing!
thread 4 is done


Using events to signal other threads

In [8]:
my_event = threading.Event()
my_thread = threading.Thread(name='wait_for_an_event', target=wait_for_an_event, args=(my_event, 5))
print("starting thread")
my_thread.start()
for i in range(5):
    time.sleep(0.5)
    print("set event")
    my_event.set()
my_thread.join()

starting thread
waiting for event
set event
got event
waiting for event
set event
got event
waiting for event
set event
got event
waiting for event
set event
got event
waiting for event
set event
got event
all events detected, thread terminating...


Using timers to run a function at some future point in time

In [9]:
def my_function(a_string):
    print(a_string)

timer_thread = threading.Timer(2, my_function,args=("time's up!",))
timer_thread.start()
print("timer_thread=", timer_thread)
timer_thread.join()   # might as well wait in this demo but obviously you might do something interesting here, or not
print("timer_thread=", timer_thread)

timer_thread= <Timer(Thread-5, started 14592)>
time's up!
timer_thread= <Timer(Thread-5, stopped 14592)>


Using barriers to ensure threads sync up where necessary

In [10]:
import datetime

def thread_a(a_barrier):
    print("thread_a started at", datetime.datetime.now())
    time.sleep(2)       # do something that needs to be done for both threads before go any further e.g. client/server
    a_barrier.wait()
    print("thread_a: ok all in sync now, we can start...")
    # do something exciting....
    return f"thread_a: time = {datetime.datetime.now()}"

def thread_b(a_barrier):
    print("thread_b started at", datetime.datetime.now())
    a_barrier.wait()
    print("thread_b: ok all in sync now, we can start...")
    # do something exciting....
    return f"thread_b: time = {datetime.datetime.now()}"

my_barrier = threading.Barrier(2)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    my_futures = [executor.submit(f, my_barrier) for f in [thread_a, thread_b]]
    for f in concurrent.futures.as_completed(my_futures):
        print(f.result())

thread_a started at 2022-11-06 11:59:36.953807
thread_b started at 2022-11-06 11:59:36.961805
thread_a: ok all in sync now, we can start...thread_b: ok all in sync now, we can start...
thread_b: time = 2022-11-06 11:59:38.967935

thread_a: time = 2022-11-06 11:59:38.968954


Define three functions that will run in separate threads

In [11]:
def thread_a(a_condition
    print("thread_a started at", datetime.datetime.now())
    with a_condition:
        print("thread_a: creating the shared resource")
        time.sleep(2)           # let's create a shared resource
        print("thread_a: resource now available, notify everyone")
        a_condition.notify_all()    # notify others shared resource is available
        print("thread_a: others now notified")
    # do something exciting....
    return f"thread_a: time = {datetime.datetime.now()}"

def thread_b(a_condition):
    print("thread_b started at", datetime.datetime.now())
    with a_condition:
        print("thread_b: waiting for shared resource")
        a_condition.wait()
        print("thread_b: we can now use shared resource")
        # do something exciting....
    return f"thread_b: time = {datetime.datetime.now()}"

def thread_c(a_condition):
    print("thread_c started at", datetime.datetime.now())
    with a_condition:
        print("thread_c: waiting for shared resource")
        a_condition.wait()
        print("thread_c: we can now use shared resource")
        # do something exciting....
    return f"thread_c: time = {datetime.datetime.now()}"

SyntaxError: '(' was never closed (330352817.py, line 1)

Using conditionals to let other threads know when they can start

In [ ]:
my_condition = threading.Condition()
my_notifier_thread = threading.Thread(target=thread_a, args=(my_condition,))
consumer_thread_b = threading.Thread(target=thread_b, args=(my_condition,))
consumer_thread_c = threading.Thread(target=thread_c, args=(my_condition,))

consumer_thread_b.start()
time.sleep(1)
consumer_thread_c.start()
time.sleep(1)
my_notifier_thread.start()